In [ ]:
## Test Team Formation framework with Freelancer Data
## Balancing Task Coverage vs. Maximum Expert Load
## Karan Vombatkere, Dec 2021

#Imports
import pandas as pd
import matplotlib.pyplot as plt
import json
import TeamFormationProblem as TFP

#### Simple synthetic Data

In [ ]:
#Preliminary Testing
task_list = [['s1', 's2', 's4', 's5'], ['s3', 's2'], ['s1', 's3', 's5'], ['s2', 's4'],['s1', 's4', 's5'], ['s3'], ['s1', 's5'], ['s2', 's4']]
expert_list = [['s2', 's3'], ['s2', 's4'], ['s3'], ['s5', 's4']]

teamFormationTest = TFP.TeamFormationProblem(task_list, expert_list, max_workload_threshold=6)
#teamFormationTest.computeTaskAssigment(lazy_eval=False)

In [ ]:
t_reg,f_reg, r_reg = teamFormationTest.computeTaskAssigment(baselines=['task_greedy'], plot_flag=False)

In [ ]:
#teamFormationTest.compareTest_Lazy_Stochastic_Assignments()
#t,f, r = teamFormationTest.compute_reverseThreshold()
#reverse_runtime, regular_runtime = teamFormationTest.compare_Methods()

#### Freelancer Dataset

In [ ]:
#Import Freelancer data
#Freelancer from DropBox link: https://www.dropbox.com/sh/8zpsi1etvvvvj5k/AAD-J9ZQmSsbnSmEILBMD9uxa/datasets/real?dl=0&subfolder_nav_tracking=1
#freelance_experts.csv and freelance_projects.csv

def extract_skills(row):
    skills = []
    for i,val in enumerate(row):
        if val == 1:
            skills.append(str(i))
    return skills            

    
def importFreelancerData(experts_filename='datasets/freelancer/freelance_experts.csv', tasks_filename='datasets/freelancer/freelance_projects.csv'):
    #Extract tasks skills as list
    freelance_tasks_df = pd.read_csv(tasks_filename, header=None)
    freelance_tasks_df['Task_Skills'] = freelance_tasks_df.apply(lambda row: extract_skills(row), axis=1)
    task_skills_list = freelance_tasks_df.Task_Skills.to_list()
    
    #Extract experts skills as list
    freelance_experts_df = pd.read_csv(experts_filename, header=None)
    freelance_experts_df['Expert_Skills'] = freelance_experts_df.apply(lambda row: extract_skills(row), axis=1)
    expert_skills_list = freelance_experts_df.Expert_Skills.to_list()

    print("Imported Freelancer dataset. Num Experts={}, Num Tasks={}".format(len(expert_skills_list),len(task_skills_list)))

    return task_skills_list, expert_skills_list
    

In [ ]:
t,e = importFreelancerData()
FreelancerTest = TFP.TeamFormationProblem(t[0:200], e[0:200], max_workload_threshold=50)

In [ ]:
a, b, c = FreelancerTest.computeTaskAssigment(baselines=['no_update_greedy'], plot_flag=True)

#### IMDB Datasets

In [ ]:
#Import IMDB Data
def importIMDBData(experts_filename, tasks_filename):
    with open(experts_filename, 'r') as f:
        expert_skills_list = json.loads(f.read())
    
    with open(tasks_filename, 'r') as f:
        task_skills_list = json.loads(f.read())

    print("Imported IMDB dataset. Num Experts={}, Num Tasks={}".format(len(expert_skills_list),len(task_skills_list)))

    return task_skills_list, expert_skills_list, 

#Run algorithm on IMDB datasets
def testIMDBDatasets():
    imdb_data_path = 'datasets/imdb/'
    movieYears = [2000, 2015, 2020]
    for y in movieYears:
        experts_file = imdb_data_path + 'imdb_experts_' + str(y) + '.txt'
        tasks_file = imdb_data_path + 'imdb_tasks_' + str(y) + '.txt'
        print("IMDB Dataset: {}, {}".format('imdb_experts_' + str(y), 'imdb_tasks_' + str(y)))

        imdb_tasks, imdb_experts = importIMDBData(experts_file, tasks_file)
        IMDBTest = TFP.TeamFormationProblem(imdb_tasks[0:2000], imdb_experts[0:2000])

        a, b, c = IMDBTest.computeTaskAssigment(baselines=['random', 'no_update_greedy'], plot_flag=True)

    return None
    

In [ ]:
testIMDBDatasets()

#### Bibsonomy Datasets

In [ ]:
#Import Bibsonomy datasets
def importBibsonomyData(experts_filename, tasks_filename):
    with open(experts_filename, 'r') as f:
        expert_skills_list = json.loads(f.read())
    
    with open(tasks_filename, 'r') as f:
        task_skills_list = json.loads(f.read())

    print("Imported Bibsonomy dataset. Num Experts={}, Num Tasks={}".format(len(expert_skills_list),len(task_skills_list)))

    return task_skills_list, expert_skills_list

#Run algorithm on Bibsonomy datasets
def testBibsonomyDatasets():
    imdb_data_path = 'datasets/bibsonomy/'
    movieYears = [2000, 2010, 2015, 2020]
    for y in movieYears:
        experts_file = imdb_data_path + 'bibsonomy_experts_' + str(y) + '.txt'
        tasks_file = imdb_data_path + 'bibsonomy_tasks_' + str(y) + '.txt'
        print("\nBibsonomy Dataset: {}, {}".format('bibsonomy_experts_' + str(y), 'bibsonomy_tasks_' + str(y)))

        bib_tasks, bib_experts = importBibsonomyData(experts_file, tasks_file)
        BibsonomyTest = TFP.TeamFormationProblem(bib_tasks[0:800], bib_experts[0:1100])

        a, b, c = BibsonomyTest.computeTaskAssigment(baselines=['no_update_greedy'], plot_flag=True)

    return None


In [ ]:
testBibsonomyDatasets()

In [ ]:
# max_threshold_arr = [5,10,40,80,100,150,200]
# rev_rt_arr, reg_rt_arr = [],[]
# for thresh in max_threshold_arr:
#     FreelancerTest = TFP.TeamFormationProblem(t[0:200], e[0:200], max_workload_threshold=thresh)
#     rev_rt, reg_rt = FreelancerTest.compare_Methods()
#     rev_rt_arr.append(rev_rt)
#     reg_rt_arr.append(reg_rt)

#Plot Runtimes
# plt.figure(figsize=(9,6))
# plt.plot(max_threshold_arr, rev_rt_arr, label='Reverse Threshold Runtime')
# plt.plot(max_threshold_arr, reg_rt_arr, label='Regular Lazy Runtime')

# title_text = 'Reverse Threshold vs. Regular Lazy runtimes'
# plt.title(title_text, fontsize=11)
# plt.xlabel('Max Threshold, T_i')
# plt.ylabel('Runtime, s')
# plt.legend(loc='lower right')
# plt.show()

#FreelancerTest.compute_reverseThreshold()
#FreelancerTest.compareTest_Lazy_Stochastic_Assignments()